In [11]:
import pandas as pd
from sqlalchemy import create_engine
import os
import json
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def load_credentials(path = "aws_rds_credentials.json"):
     with open(path, 'r') as file:
          config = json.load(file)

     # set up credentials
     for key in config.keys():
          os.environ[key] = config[key]

     return

time_interval = 90 #days

load_credentials()

aws_rds_url = f"postgresql://{os.environ['user']}:{os.environ['password']}@{os.environ['host']}:{os.environ['port']}/{os.environ['database']}?sslmode=require"

engine = create_engine(aws_rds_url)
sql_query = f"""SELECT brand_title, price_numeric, status, catalog_id, size_title, color1_id
               FROM public.tracking_staging 
               WHERE date >= CURRENT_DATE - INTERVAL '{time_interval} days'
               """
data = pd.read_sql(sql_query, engine)
data

,brand_title,price_numeric,status,catalog_id,size_title,color1_id
0,Diesel,15.0,Bom,1845.0,XS / 34 / 6,9.0
1,Calvin Klein,15.0,Muito bom,1845.0,L / 40 / 12,9.0
2,Stradivarius,5.0,Muito bom,1079.0,S / 36 / 8,4.0
3,Stradivarius,7.0,Novo sem etiquetas,1043.0,M / 38 / 10,2.0
4,Pull & Bear,5.0,Novo sem etiquetas,190.0,S / 36 / 8,4.0
...,...,...,...,...,...,...
404651,CUIR,14.0,Satisfatório,1238.0,43,2.0
404652,Meltin Pot,14.0,Bom,1844.0,L / 40 / 12,9.0
404653,Diesel,18.0,Muito bom,1845.0,M / 38 / 10,1.0
404654,Menglu,3.0,Bom,1066.0,Tamanho único,3.0


In [16]:
# inputs: count products, median price, std dev price, nunique catalog_id, ununique color1_id, ununique size_title
# Group by brand_title and calculate various statistics
brand_stats = data.groupby('brand_title').agg({
    'price_numeric': ['count', 'median', 'std'],
    'catalog_id': 'nunique',
    'color1_id': 'nunique',
    'size_title': 'nunique'
})

# Rename columns for clarity
brand_stats.columns = ['product_count', 'price_median', 'price_std', 'catalog_id_unique', 'color1_id_unique', 'size_title_unique']

brand_stats = brand_stats[brand_stats["product_count"] > 300]
brand_stats

,product_count,price_median,price_std,catalog_id_unique,color1_id_unique,size_title_unique
brand_title,,,,,,
,64286,5.0,11.804352,610,49,164
ASOS,642,7.0,9.817804,62,22,26
Abercrombie & Fitch,414,8.0,9.914441,32,17,16
Alcott,528,5.0,7.719637,32,20,15
Armand Thiery,319,7.0,6.475775,33,19,14
...,...,...,...,...,...,...
Vintage Dressing,1221,20.0,15.690781,68,30,17
Zara,14902,7.0,10.487872,300,50,78
adidas,6405,15.0,45.471215,139,39,81


In [15]:
pca = PCA(n_components=2)
X_2d = pca.fit_transform(brand_stats)

# Assign cluster labels to the transformed data
data_with_labels = pd.DataFrame(X_2d, index=brand_stats.index, columns=['PC1', 'PC2'])

# Plot the transformed data with colors based on cluster labels
plt.figure(figsize=(8, 6))
for cluster_label in data_with_labels['cluster_label'].unique():
    plt.scatter(data_with_labels.loc[data_with_labels['cluster_label'] == cluster_label, 'PC1'],
                data_with_labels.loc[data_with_labels['cluster_label'] == cluster_label, 'PC2'],
                label=f'Cluster {cluster_label}')
plt.title('PCA to 2 Dimensions with Cluster Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().